In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
np.random.seed(333)
import joblib
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow import keras
from tensorflow.keras.layers import (
    Conv1D,
    Dense,
    Dropout,
    Flatten,
    MaxPooling1D
)

from tensorflow.keras.metrics import RootMeanSquaredError 
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt


import matplotlib.pyplot as plt

In [3]:
import pandas as pd
df_train = pd.read_csv(r"drive/My Drive/classification_model/df_training.csv")
df_test = pd.read_csv(r"drive/My Drive/classification_model/df_testing.csv")
df_validation = pd.read_csv(r"drive/My Drive/classification_model/df_validation.csv")

In [4]:
#pd.set_option('display.max_rows', 1000)
df_train.isna().sum()

Unnamed: 0        0
filename          0
time              0
video_b           0
video_g           0
                 ..
tonnetz_mean      0
tonnetz_std       0
tonnetz_median    0
tonnetz_len       0
label             0
Length: 129, dtype: int64

In [5]:
df_validation.rename({'mfcc_sum_y': 'mfcc_sum'}, axis=1, inplace=True)  # new method
df_validation.drop(["mfcc_sum_x"] , axis=1, inplace=True )     
df_test.rename({'mfcc_sum_y': 'mfcc_sum'}, axis=1, inplace=True)  # new method
df_test.drop(["mfcc_sum_x"] , axis=1, inplace=True )             
#df.rename({'a': 'X', 'b': 'Y'}, axis=1, inplace=True)  # new method

In [6]:
#df_validation
df_train = df_train.iloc[: , 1:]
df_test = df_test.iloc[: , 1:]
df_validation = df_validation.iloc[: , 1:]

In [7]:
df_train = df_train.iloc[: , 1:]
df_test = df_test.iloc[: , 1:]
df_validation = df_validation.iloc[: , 1:]

In [8]:
df_train.drop(["chroma_stft","spectral_contrast"	,"zero",	"chroma_cqt",	"chroma_cens",	"melspectrogram",	"mfcc",	"rms"	,"spectral_centroid",	"spectral_bandwidth",		"spectral_flatness",	"spectral_rolloff",	"poly_features",	"tonnetz"], axis=1, inplace=True)
df_test.drop(["chroma_stft","spectral_contrast"	,"zero",	"chroma_cqt",	"chroma_cens",	"melspectrogram",	"mfcc",	"rms"	,"spectral_centroid",	"spectral_bandwidth",		"spectral_flatness",	"spectral_rolloff",	"poly_features",	"tonnetz"], axis=1, inplace=True)
df_validation.drop(["chroma_stft","spectral_contrast"	,"zero",	"chroma_cqt",	"chroma_cens",	"melspectrogram",	"mfcc",	"rms"	,"spectral_centroid",	"spectral_bandwidth",		"spectral_flatness",	"spectral_rolloff",	"poly_features",	"tonnetz"], axis=1, inplace=True)



In [9]:
df_train["label"].replace({"animated": 1, "non-animated": 0}, inplace=True)
df_test["label"].replace({"animated": 1, "non-animated": 0}, inplace=True)
df_validation["label"].replace({"animated": 1, "non-animated": 0}, inplace=True)


In [10]:
df_train.reset_index(inplace=True,drop=True)
df_test.reset_index(inplace=True,drop=True)
df_validation.reset_index(inplace=True,drop=True)

In [11]:
df_train.drop(["chroma_cqt_max", "chroma_stft_max","mfcc_sum" ], axis=1, inplace=True)  
df_test.drop(["chroma_cqt_max", "chroma_stft_max","mfcc_sum" ], axis=1, inplace=True)  
df_validation.drop(["chroma_cqt_max", "chroma_stft_max","mfcc_sum" ], axis=1, inplace=True)  

df_train.reset_index(inplace=True,drop=True)
df_test.reset_index(inplace=True,drop=True)
df_validation.reset_index(inplace=True,drop=True)

In [12]:
df_train.shape

(999, 110)

In [13]:
 dff=df_train.append(df_test)

In [14]:
dfff=dff.append(df_validation)

In [15]:
dfff.shape

(1394, 110)

In [16]:
dfff.isna().sum()

time              0
video_b           0
video_g           0
video_r           0
video_h           0
                 ..
tonnetz_mean      0
tonnetz_std       0
tonnetz_median    0
tonnetz_len       0
label             0
Length: 110, dtype: int64

In [17]:
labels=dfff['label']
features = dfff.iloc[:,0:109]
labelss=np.ravel(labels)

In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(features)

features = scaler.transform(features)



In [19]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np

# Model configuration



verbosity = 1
num_folds = 10

inputs = features
targets = labelss

# Determine shape of the data
input_shape=[(features.shape)[1]]



# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []


# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
  model=Sequential([
    Dense(109, activation='relu',input_shape=input_shape,kernel_regularizer=keras.regularizers.l2(l=0.001)),
    #Dense(64,activation='relu',kernel_regularizer=keras.regularizers.l2(l=0.001)),
    Dense(128,activation='relu'),
    Dense(128,activation='relu'),
    Dense(64,activation='relu'),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(8,activation='relu'),
    Dense(1,activation='sigmoid')

  ])

  # Compile the model
  model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
 
  history=model.fit(inputs[train], targets[train], epochs=120, batch_size=500, verbose=1)

  # Fit data to model

  # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/120
3/3 [==============================] - 1s 7ms/step - loss: 0.8174 - accuracy: 0.5024
Epoch 2/120
3/3 [==============================] - 0s 7ms/step - loss: 0.8104 - accuracy: 0.5024
Epoch 3/120
3/3 [==============================] - 0s 7ms/step - loss: 0.8042 - accuracy: 0.5024
Epoch 4/120
3/3 [==============================] - 0s 6ms/step - loss: 0.7985 - accuracy: 0.5024
Epoch 5/120
3/3 [==============================] - 0s 7ms/step - loss: 0.7932 - accuracy: 0.5024
Epoch 6/120
3/3 [==============================] - 0s 9ms/step - loss: 0.7883 - accuracy: 0.5024
Epoch 7/120
3/3 [==============================] - 0s 8ms/step - loss: 0.7835 - accuracy: 0.5040
Epoch 8/120
3/3 [==============================] - 0s 7ms/step - loss: 0.7790 - accuracy: 0.5032
Epoch 9/120
3/3 [==============================] - 0s 7ms/step - loss: 0.7746 - accuracy: 0.5032
Epoch 10/120
3/3 [============

In [20]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_72 (Dense)            (None, 109)               11990     
                                                                 
 dense_73 (Dense)            (None, 128)               14080     
                                                                 
 dense_74 (Dense)            (None, 128)               16512     
                                                                 
 dense_75 (Dense)            (None, 64)                8256      
                                                                 
 dense_76 (Dense)            (None, 64)                4160      
                                                                 
 dense_77 (Dense)            (None, 32)                2080      
                                                                 
 dense_78 (Dense)            (None, 8)                